# **Criando a Rede**

In [ ]:
import torch
from torch import nn

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)

cpu


Trabalhando com dataset de classificação de vinhos

In [ ]:
from sklearn import datasets

wine = datasets.load_wine()
data = wine.data
target = wine.target

#178 amostras com 13 características
print(data.shape, target.shape)
#13 características, 3 classes
print(wine.feature_names, wine.target_names)

(178, 13) (178,)
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline'] ['class_0' 'class_1' 'class_2']


In [ ]:
class WineClassifier(nn.Module):

  def __init__(self, input_size, hidden_size, out_size):
    super(WineClassifier, self).__init__()

    self.hidden = nn.Linear(input_size, hidden_size)
    self.relu   = nn.ReLU()
    self.out    = nn.Linear(hidden_size, out_size)
    #A softmax é responsável pela distribuição de probabilidades
    self.softmax = nn.Softmax()

  def forward(self, X):
      feature = self.relu(self.hidden(X))
      output = self.softmax(self.out(feature))

      return output

input_size = data.shape[1] #Número de características
hidden_size = 32 # 32 neurônios na camada escondida, euq ue decidi o valor
out_size = len(wine.target_names) #No caso, serão 3
#Criando a rede
net = WineClassifier(input_size, hidden_size, out_size).to(device) #cast na GPU

In [ ]:
print(net)

WineClassifier(
  (hidden): Linear(in_features=13, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=None)
)


# **Classificação**

Instanciando a Função de Perda. Trata-se de um problema de classificação com 3 classes. Neste caso, a Cross Entropy é a função recomendada, que no Pytorch recebe o nome de *CrossEntropyLoss*

In [ ]:
criterion = nn.CrossEntropyLoss().to(device) #cast na GPU

In [ ]:
#tranformando os dados de arrays do numpy para Tensores
#transformanfo o Xtns de float64 para float32
Xtns = torch.from_numpy(data).float()
Ytns = torch.from_numpy(target)

#cast na GPU
Xtns = Xtns.to(device)
Ytns = Ytns.to(device)

print(Xtns.dtype, Ytns.dtype)

torch.float32 torch.int64


In [ ]:
#Saída da rede
#Exemplo de uma  amostra
pred = net(Xtns[0].unsqueeze(0))

print(Xtns[0].unsqueeze(0).shape)
print(pred.shape)
#Previu a classe 1 na saída
print(pred)

torch.Size([1, 13])
torch.Size([1, 3])
tensor([[3.1550e-05, 9.9997e-01, 0.0000e+00]], grad_fn=<SoftmaxBackward0>)


<ipython-input-24-77b5800f2961>:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(self.out(feature))


In [ ]:
pred = net(Xtns)

<ipython-input-24-77b5800f2961>:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(self.out(feature))


In [ ]:
print(pred.shape, Ytns.shape)
#No pred.shape, número de amostras por número de classes
#No Ytns.shape, número de amostras

print(pred[0].data, Ytns[0].data)

torch.Size([178, 3]) torch.Size([178])
tensor([3.1550e-05, 9.9997e-01, 0.0000e+00]) tensor(0)


In [ ]:
loss = criterion(pred, Ytns)
print(loss)
#O loss calcula a perda média entre os valores

tensor(1.3138, grad_fn=<NllLossBackward0>)


# **Regressão**

Trabalahndo com o dataset de diabetes

In [ ]:
from sklearn import datasets

diabetes = datasets.load_diabetes()
data = diabetes.data
target = diabetes.target

print(data.shape)

#442 amostras, com 10 características cada amostra
print(data[0], target[0])
#442 rótulos, somente 1 saída
print(target[0])

(442, 10)
[ 0.03807591  0.05068012  0.06169621  0.02187239 -0.0442235  -0.03482076
 -0.04340085 -0.00259226  0.01990749 -0.01764613] 151.0
151.0


Implementando o MLP

In [ ]:
class DiabetesClassifier(nn.Module):

  def __init__(self, input_size, hidden_size, out_size):
    super(DiabetesClassifier, self).__init__()

    self.hidden = nn.Linear(input_size, hidden_size)
    self.relu   = nn.ReLU()
    self.out    = nn.Linear(hidden_size, out_size)
    #A softmax é responsável pela distribuição de probabilidades
    self.softmax = nn.Softmax()

  def forward(self, X):
      feature = self.relu(self.hidden(X))
      output = self.softmax(self.out(feature))

      return output

input_size = data.shape[1] #Número de características
hidden_size = 32 # 32 neurônios na camada escondida, euq ue decidi o valor
out_size = 1 #progressão da diabetes, 1 neurônio
#Criando a rede
net = DiabetesClassifier(input_size, hidden_size, out_size).to(device) #cast na GPU

Para solucionar problemas de regressão, as funções de perda correspondetes esperam que ambos o rótulo e a predição tenham a mesma dimensionalidade. Não se trata de um problema categórico.


Portanto, vamos simular um problema de regressão e aplicar o MSELoss

In [ ]:
criterion = nn.MSELoss().to(device)

#Cast na GPU

Xtns = torch.from_numpy(data).float().to(device)
Ytns = torch.from_numpy(target).float().to(device)

print(Xtns.shape)
print(Ytns.shape)

torch.Size([442, 10])
torch.Size([442])


In [ ]:
pred = net(Xtns)

loss = criterion(pred.squeeze(), Ytns)
print(loss.data)
#media das distancias com base no MSE

tensor(28771.2148)


<ipython-input-43-921ea0c14561>:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(self.out(feature))
